In [ ]:
%pip install flask --break-system-packages
%pip install flask-restx  --break-system-packages
%pip install flask flask-sqlalchemy   --break-system-packages
%pip install  mysql-connector-python --break-system-packages
%pip install sendgrid --break-system-packages

In [ ]:
from flask import Flask
from flask_cors import CORS
from flask_restx import Api, Resource, reqparse, fields
from werkzeug.datastructures import FileStorage
from werkzeug.serving import run_simple
import os

app = Flask(__name__)
CORS(app)
api = Api(app, title='Image Upload API', description='API for uploading images with metadata')

# Define upload folder
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure upload directory exists
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

# Define allowed file extensions
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Create namespace
ns = api.namespace('upload', description='Image upload operations')

# Define parser for file upload and additional fields
upload_parser = reqparse.RequestParser()
upload_parser.add_argument('image', 
                          type=FileStorage, 
                          location='files', 
                          required=True, 
                          help='Image file is required')
upload_parser.add_argument('name', 
                         type=str, 
                         required=True, 
                         help='Name is required')
upload_parser.add_argument('email', 
                         type=str, 
                         required=True, 
                         help='Email is required')

# Define response model for Swagger documentation
upload_response = api.model('UploadResponse', {
    'message': fields.String,
    'filename': fields.String,
    'path': fields.String,
    'name': fields.String,
    'email': fields.String
})

@ns.route('/image')
class ImageUpload(Resource):
    @ns.expect(upload_parser)
    @ns.marshal_with(upload_response)
    def post(self):
        """Upload an image file with name and email"""
        args = upload_parser.parse_args()
        image_file = args['image']
        name = args['name']
        email = args['email']
        
        # Check if the file has an allowed extension
        if image_file and allowed_file(image_file.filename):
            # Secure the filename
            filename = image_file.filename
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            
            try:
                # Save the file
                image_file.save(filepath)
                return {
                    'message': 'Image uploaded successfully',
                    'filename': filename,
                    'path': filepath,
                    'name': name,
                    'email': email
                }, 201
            except Exception as e:
                return {
                    'message': f'Failed to upload image: {str(e)}'
                }, 500
        else:
            return {
                'message': 'Invalid file type. Allowed types: png, jpg, jpeg, gif'
            }, 400
 
# Using run_simple instead of app.run()
run_simple('localhost', 5000, app, use_reloader=False, use_debugger=True)


In [ ]:
from flask import Flask
from flask_cors import CORS
from flask_restx import Api, Resource, reqparse, fields
from werkzeug.serving import run_simple
import smtplib
from email.mime.text import MIMEText

app = Flask(__name__)
CORS(app)
api = Api(app, title='Email API', description='API for sending emails')

# Email configuration
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
FROM_EMAIL = 'fuzumoe@gmail.com'  # Your email
EMAIL_PASSWORD = '5ZzF97RgqUC6dE'  # Your password/App Password

def send_email(to_email, name):
    """Send confirmation email using SMTP"""
    subject = 'Email Confirmation'
    body = f"""
    Hello {name},
    This is a confirmation email from our service!
    Thank you for using it.
    """
    
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = FROM_EMAIL
    msg['To'] = to_email

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()  # Enable TLS
        server.login(FROM_EMAIL, EMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        print(f"Email sent to {to_email}")
        return True
    except Exception as e:
        print(f"Failed to send email: {str(e)}")
        return False

# Create namespace
ns = api.namespace('email', description='Email sending operations')

# Define parser for email fields
email_parser = reqparse.RequestParser()
email_parser.add_argument('name', 
                         type=str, 
                         required=True, 
                         help='Name is required')
email_parser.add_argument('email', 
                         type=str, 
                         required=True, 
                         help='Email is required')

# Define response model for Swagger documentation
email_response = api.model('EmailResponse', {
    'message': fields.String,
    'name': fields.String,
    'email': fields.String,
    'email_sent': fields.Boolean
})

@ns.route('/send')
class EmailSender(Resource):
    @ns.expect(email_parser)
    @ns.marshal_with(email_response)
    def post(self):
        """Send an email with name and email provided"""
        args = email_parser.parse_args()
        name = args['name']
        email = args['email']
        
        try:
            # Send confirmation email
            email_sent = send_email(email, name)
            
            return {
                'message': 'Email processed successfully',
                'name': name,
                'email': email,
                'email_sent': email_sent
            }, 200
        except Exception as e:
            return {
                'message': f'Failed to process email: {str(e)}'
            }, 500
 
run_simple('localhost', 5000, app, use_reloader=False, use_debugger=True)

In [ ]:
from flask import Flask
from flask_cors import CORS
from flask_restx import Api, Resource, reqparse, fields
from werkzeug.serving import run_simple
import smtplib
from email.mime.text import MIMEText
import os
import secrets
import string

app = Flask(__name__)
CORS(app)
api = Api(app, title='User Creation API', description='API for creating users and sending passwords')

# Email configuration (using environment variables for security)
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
FROM_EMAIL = os.environ.get('EMAIL_ADDRESS', 'your_email@gmail.com')  # Set via env var
EMAIL_PASSWORD = os.environ.get('EMAIL_PASSWORD', 'your_password')    # Set via env var

# In-memory storage for users (replace with a database in production)
users = {}

def generate_random_password(length=12):
    """Generate a random password"""
    alphabet = string.ascii_letters + string.digits + string.punctuation
    return ''.join(secrets.choice(alphabet) for _ in range(length))

def send_email(to_email, name, password):
    """Send the generated password to the user's email with HTML and CSS"""
    subject = 'Your New Account Password'
    html_body = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <style>
            body {{
                font-family: Arial, sans-serif;
                background-color: #f4f4f4;
                margin: 0;
                padding: 0;
            }}
            .container {{
                width: 100%;
                max-width: 600px;
                margin: 20px auto;
                background-color: #ffffff;
                border-radius: 8px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                padding: 20px;
            }}
            .header {{
                background-color: #4CAF50;
                color: white;
                padding: 10px;
                text-align: center;
                border-radius: 8px 8px 0 0;
            }}
            .content {{
                padding: 20px;
                line-height: 1.6;
            }}
            .password-box {{
                background-color: #f9f9f9;
                border: 1px solid #ddd;
                padding: 10px;
                font-size: 16px;
                color: #333;
                text-align: center;
                margin: 10px 0;
            }}
            .footer {{
                text-align: center;
                font-size: 12px;
                color: #777;
                padding-top: 10px;
            }}
            a {{
                color: #4CAF50;
                text-decoration: none;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h2>Welcome, {name}!</h2>
            </div>
            <div class="content">
                <p>Your account has been created successfully!</p>
                <p>Here is your password:</p>
                <div class="password-box">{password}</div>
                <p>Please keep it safe and <a href="#">change it</a> after logging in.</p>
            </div>
            <div class="footer">
                <p>Thank you for using our service &copy; 2025</p>
            </div>
        </div>
    </body>
    </html>
    """
    
    msg = MIMEText(html_body, 'html')  # Set MIME type to 'html'
    msg['Subject'] = subject
    msg['From'] = FROM_EMAIL
    msg['To'] = to_email

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()  # Enable TLS
        server.login(FROM_EMAIL, EMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        print(f"Email sent to {to_email}")
        return True
    except Exception as e:
        print(f"Failed to send email: {str(e)}")
        return False

# Create namespace
ns = api.namespace('users', description='User management operations')

# Define parser for user creation
user_parser = reqparse.RequestParser()
user_parser.add_argument('name', 
                        type=str, 
                        required=True, 
                        help='Name is required')
user_parser.add_argument('email', 
                        type=str, 
                        required=True, 
                        help='Email is required')

# Define response model for Swagger documentation
user_response = api.model('UserResponse', {
    'message': fields.String,
    'name': fields.String,
    'email': fields.String,
    'email_sent': fields.Boolean
})

@ns.route('/create')
class UserCreate(Resource):
    @ns.expect(user_parser)
    @ns.marshal_with(user_response)
    def post(self):
        """Create a new user with a random password and send it via email"""
        args = user_parser.parse_args()
        name = args['name']
        email = args['email']

        # Check if email already exists
        if email in users:
            return {
                'message': 'User with this email already exists'
            }, 400

        # Generate random password
        password = generate_random_password()

        # Store user (in memory for this example)
        users[email] = {
            'name': name,
            'password': password  # In production, hash this!
        }

        try:
            # Send email with the password
            email_sent = send_email(email, name, password)
            
            return {
                'message': 'User created successfully',
                'name': name,
                'email': email,
                'email_sent': email_sent
            }, 201
        except Exception as e:
            # Clean up if email fails (optional)
            users.pop(email, None)
            return {
                'message': f'Failed to create user: {str(e)}'
            }, 500
 
run_simple('localhost', 5000, app, use_reloader=False, use_debugger=True)